# Load model

In [ ]:
import torch
import torch
from tqdm import tqdm
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from operator import itemgetter
from ENet.DenseCRF import dense_crf_from_probabilities
import cv2
import pickle
# Import your custom modules (make sure these are accessible in your notebook)
from ENet.ENet import ENet
from ENet.dataset import SliceDataset
from ENet.utils import dice_coef, probs2one_hot, probs2class, class2one_hot
import seaborn as sns
import pandas as pd
from collections import defaultdict


# Define custom palette
custom_palette = {
    'roze': '#eb8fd8',
    'groen': '#b9d4b4',
    'paars': '#ba94e9',
    'blue': '#4C8BE2',
    'orange': '#E27A3F',
    'grey_light': '#1F3240',
    'grey_dark': '#16242F'
}

# Define custom themes
def set_custom_dark_theme():
    sns.set_context('notebook', font_scale=1.2)
    sns.set_style({
        'axes.facecolor': custom_palette['grey_light'],
        'axes.edgecolor': 'white',
        'axes.labelcolor': 'white',
        'xtick.color': 'white',
        'ytick.color': 'white',
        'grid.color': 'white',
        'figure.facecolor': custom_palette['grey_light'],
        'text.color': 'white'
    })
    sns.set_palette([
        custom_palette['roze'],
        custom_palette['groen'],
        custom_palette['paars'],
        custom_palette['blue'],
        custom_palette['orange']
    ])

def set_custom_light_theme():
    sns.set_context('notebook', font_scale=1.2)
    sns.set_style({
        'axes.facecolor': 'white',
        'axes.edgecolor': 'black',
        'axes.labelcolor': 'black',
        'xtick.color': 'black',
        'ytick.color': 'black',
        'grid.color': custom_palette["grey_light"],
        'figure.facecolor':'white',
        'text.color': 'black'
    })
    sns.set_palette([
        custom_palette['roze'],
        custom_palette['groen'],
        custom_palette['paars'],
        custom_palette['blue'],
        custom_palette['orange']
    ])


In [ ]:
import json

# nnunet_json_files = {
#     'nnUNet 2D best model': 'model_examples/2d/combined/Combined_Dice_CE_loss_summary.json',
#     'nnUNet 3D best model': 'model_examples/3d/combined/3d_lowres_Dataset060_SegTHOR_Combined_Dice_CE_loss_summary.json'
# }

# nnunet_json_files = {
#     "Focal Loss": "model_examples/3d/corrected_GT/Focal/3d_lowres_nnUNetTrainerFocalLoss_summary.json",
#     "Tversky Loss": "model_examples/3d/corrected_GT/Tversky/3d_lowres_Tversky_loss_summary.json",
#     "Dice Loss": "model_examples/3d/corrected_GT/Dice_loss/3d_lowres_Dice_loss_summary.json",
#     "Combined Loss": "model_examples/3d/corrected_GT/Dice_CE_loss/3d_lowres_Dice_CE_loss_summary.json",
#     'Cross Entropy Loss': 'model_examples/3d/corrected_GT/CE_loss/3d_lowres_CE_loss_summary.json'
# }

# nnunet_json_files = {
#     "Noise": "model_examples/3d/noise/3d_lowres_Dataset059_SegTHOR_Noise_Dice_CE_loss_summary.json",
#     "Affine": "model_examples/3d/affine/3d_lowres_Dataset057_SegTHOR_Affine_Dice_CE_loss_summary.json",
#     "Elastic": "model_examples/3d/elastic/3d_lowres_Dataset058_SegTHOR_Elastic_Dice_CE_loss_summary.json",
#     "All augmentations": "model_examples/3d/combined/3d_lowres_Dataset060_SegTHOR_Combined_Dice_CE_loss_summary.json",
# }

# nnunet_json_files = {
#     'no CRF': 'model_examples/3d/combined/3d_lowres_Dataset060_SegTHOR_Combined_Dice_CE_loss_summary.json',
#     'CRF': 'model_examples/3d/CRF/CRF_summary.json'
# }

model_palette = {
    'nnUNet 2D best model' : custom_palette['roze'],
    'ENet best model' : custom_palette["groen"],
    'nnUNet 3D best model' : custom_palette["paars"],
    'ENet best model + crf' : custom_palette["blue"],
    'no CRF': custom_palette["paars"],
    'CRF': custom_palette["orange"],
    'Cross Entropy Loss': custom_palette['roze'],
    "Dice Loss": custom_palette['groen'],
    "Focal Loss": custom_palette['paars'],
    "Tversky Loss": custom_palette['blue'],
    "Combined Loss": custom_palette['orange'],
    "Noise": custom_palette['roze'],
    "Affine": custom_palette['groen'],
    "Elastic": custom_palette['paars'],
    "All augmentations": custom_palette['blue'],
    
}
    


# 
# 
# # Prepare a list to hold all data
all_data = []
# 
# # Load nnUNet data
# for model_name, json_file in nnunet_json_files.items():
#     # Load the json file
#     with open(json_file, 'r') as f:
#         data = json.load(f)
# 
#     # For the metrics 'Dice' and 'Hausdorff'
#     for metric_name in ['Dice']:
#         # Collect metric values per case
#         for case in data['metric_per_case']:
#             metrics = case['metrics']
#             # Collect metrics per class
#             values = []
#             for class_idx_str, class_metrics in metrics.items():
#                 if class_idx_str == '0':
#                     print("Background class is present")
#                     continue  # Skip background
#                 value = class_metrics.get(metric_name, None)
#                 if value is not None:
#                     values.append(value)
#             # Compute average over classes
#             if values:
#                 avg_value = np.nanmean(values)
#                 all_data.append({
#                     'Model': model_name,
#                     'Metric': metric_name,
#                     'Value': avg_value
#                 })
            
            
# Paths to your ENet pickle files for the best model
enet_metrics_files = {
    "Focal Loss": "3D_focal_loss_10-23(15.08)/metrics3D.pkl",
    "Tversky Loss": "3D_tversky_loss_10-23(14.45)/metrics3D.pkl",
    "Dice Loss": "3D_dice_loss_10-23(14.52)/metrics3D.pkl",
    "Combined Loss": "3D_combined_loss_10-23(15.02)/metrics3D.pkl",
    'Cross Entropy Loss': '3D_ce_loss_10-23(15.14)/metrics3D.pkl'
}

# Load ENet data
for model_name, metrics_file in enet_metrics_files.items():
    # Load the metrics
    metrics = pickle.load(open(metrics_file, "rb"))
    for metric_name in ['3D Dice']:
        # Extract the metric values
        metric_values = np.array(metrics[metric_name])  # Shape: (num_samples, num_classes)
        num_samples, num_classes = metric_values.shape
        # Exclude background class (assume class 0 is background)
        values = metric_values[:, 1:]  # Shape: (num_samples, num_classes -1)
        # Compute the average over classes (axis=1)
        avg_values = np.nanmean(values, axis=1)  # Shape: (num_samples,)
        for value in avg_values:
            all_data.append({
                'Model': model_name,
                'Metric': 'Dice' if metric_name == '3D Dice' else 'Hausdorff',
                'Value': value
            })

        
# Create a DataFrame
df = pd.DataFrame(all_data)


In [ ]:

set_custom_light_theme()
    



# Create violin plots for each metric
for metric in ['Dice', 'Hausdorff']:
    plt.figure(figsize=(10, 6))
    ax = sns.violinplot(
        data=df[df['Metric'] == metric],
        x='Model',
        y='Value',
        palette=model_palette,
        inner=None  # Remove the default inner representation
    )

    # Calculate the mean values for each model
    means = df[df['Metric'] == metric].groupby('Model')['Value'].mean()

    # Get the positions of the x-ticks (models)
    x_positions = []
    labels = []
    for tick, label in zip(ax.get_xticks(), ax.get_xticklabels()):
        x_positions.append(tick)
        labels.append(label.get_text())

    # Get the y-axis limits to calculate an appropriate offset
    ymin, ymax = ax.get_ylim()
    y_range = ymax - ymin
    y_offset = y_range * 0.01  # Adjust as needed

    # Overlay the mean values on the violin plot
    for x, model_name in zip(x_positions, labels):
        mean_value = means[model_name]
        # Plot the mean point
        ax.scatter(x, mean_value, color='black', marker='o', s=100, zorder=3, label='Mean' if x == x_positions[0] else "")
        # Annotate the mean value, adjust position to avoid overlap
        ax.text(
            x, mean_value + y_offset,  # Move text slightly above the mean point
            f'{mean_value:.2f}',
            color='black',
            ha='center',
            va='bottom',
            fontsize=12
        )

    # Add legend only once
    handles, labels_legend = ax.get_legend_handles_labels()
    if 'Mean' in labels_legend:
        ax.legend(handles=[handles[labels_legend.index('Mean')]], labels=['Mean'], loc='upper right')

    plt.title(f"Comparison of {metric} Across Best Models (Average over Classes, excl. background)", fontsize=14)
    plt.ylabel(metric, fontsize=12)
    plt.xlabel('Model', fontsize=12)
    plt.xticks(rotation=45, fontsize=12)
    plt.tight_layout()
    plt.savefig(f'{metric}_Comparison.png', dpi=300)
    plt.show()

# Heatmap

In [ ]:
import json
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Define class names
class_names = ['Background', 'Esophagus', 'Heart', 'Trachea', 'Aorta']

# Define custom palette (adjust colors as needed)
custom_palette = {
    'Architecture': {
        'ENet': '#eb8fd8',       # Pink
        'nnUNet 2D': '#b9d4b4',  # Green
        'nnUNet 3D': '#ba94e9'   # Purple
    },
    'Augmentation': {
        'None': '#4C8BE2',        # Blue
        'Elastic': '#E27A3F',     # Orange
        'Noise': '#1F3240',       # Dark Grey
        'Affine': '#16242F',      # Darker Grey
        'Combined': '#FFB6C1'     # Light Pink
    },
    'Loss': {
        'Dice Loss': '#FFD700',       # Gold
        'CE Loss': '#ADFF2F',         # Green Yellow
        'Focal Loss': '#FF69B4',      # Hot Pink
        'Dice + CE Loss': '#20B2AA',  # Light Sea Green
        'Tversky Loss': '#9370DB'     # Medium Purple
    },
    'CRF': {
        'Without CRF': '#A9A9A9',  # Dark Grey
        'With CRF': '#000000'      # Black
    }
}

# Initialize an empty list to hold all data
all_data = []

# Define the configurations for your models
# Define the configurations for your models
models_config = [
    # Example model configurations
    {
        'Model Name': 'ENet',
        'Architecture': 'ENet',
        'Augmentation': 'Combined',
        'Loss': 'Dice + CE Loss',
        'CRF': 'Without CRF',
        'Metrics File': '3D_combineddata_10-23(15.37)/metrics3D.pkl',
        'Metrics Type': 'ENet'
    },
    {
        'Model Name': 'nnUNet 2D',
        'Architecture': 'nnUNet 2D',
        'Augmentation': 'Combined',
        'Loss': 'Dice + CE Loss',
        'CRF': 'Without CRF',
        'Metrics File': 'model_examples/2d/combined/Combined_Dice_CE_loss_summary.json',
        'Metrics Type': 'nnUNet'
    },
    {
        'Model Name': 'nnUNet 3D',
        'Architecture': 'nnUNet 3D',
        'Augmentation': 'Combined',
        'Loss': 'Dice + CE Loss',
        'CRF': 'Without CRF',
        'Metrics File': 'model_examples/3d/combined/3d_lowres_Dataset060_SegTHOR_Combined_Dice_CE_loss_summary.json',
        'Metrics Type': 'nnUNet'
    },
    {
        'Model Name': 'nnUNet 3D',
        'Architecture': 'nnUNet 3D',
        'Augmentation': 'Combined',
        'Loss': 'Dice + CE Loss',
        'CRF': 'Without CRF',
        'Metrics File': 'model_examples/3d/combined/3d_lowres_Dataset060_SegTHOR_Combined_Dice_CE_loss_summary.json',
        'Metrics Type': 'nnUNet'
    },
    # Add other models with different configurations
]

# Load data for each model
for config in models_config:
    model_name = config['Model Name']
    architecture = config['Architecture']
    augmentation = config['Augmentation']
    loss = config['Loss']
    crf = config['CRF']
    metrics_file = config['Metrics File']
    metrics_type = config['Metrics Type']
    
    # Load the metrics
    if metrics_type == 'ENet':
        # ENet metrics are stored in pickle files
        metrics = pickle.load(open(metrics_file, "rb"))
        for metric_name in ['3D Dice', '3D Hausdorff distance']:
            # Extract the metric values
            metric_values = np.array(metrics[metric_name])  # Shape: (num_samples, num_classes)
            num_samples, num_classes = metric_values.shape
            # Exclude background class (assume class 0 is background)
            values = metric_values[:, 1:]  # Shape: (num_samples, num_classes -1)
            # Compute the average over classes (axis=1)
            avg_values = np.nanmean(values, axis=1)  # Shape: (num_samples,)
            # Compute mean over samples
            mean_metric_value = np.nanmean(avg_values)
            all_data.append({
                'Model Name': model_name,
                'Architecture': architecture,
                'Augmentation': augmentation,
                'Loss': loss,
                'CRF': crf,
                'Metric': 'Dice' if metric_name == '3D Dice' else 'Hausdorff',
                'Value': mean_metric_value
            })
    elif metrics_type == 'nnUNet':
        # nnUNet metrics are stored in JSON files
        with open(metrics_file, 'r') as f:
            data = json.load(f)
        for metric_name in ['Dice', 'Hausdorff']:
            # Collect metric values per case
            avg_values = []
            for case in data['metric_per_case']:
                metrics_case = case['metrics']
                # Collect metrics per class
                values = []
                for class_idx_str, class_metrics in metrics_case.items():
                    if class_idx_str == '0':
                        continue  # Skip background
                    value = class_metrics.get(metric_name, None)
                    if value is not None:
                        values.append(value)
                # Compute average over classes for this case
                if values:
                    avg_value_case = np.nanmean(values)
                    avg_values.append(avg_value_case)
            # Compute mean over samples
            if avg_values:
                mean_metric_value = np.nanmean(avg_values)
                all_data.append({
                    'Model Name': model_name,
                    'Architecture': architecture,
                    'Augmentation': augmentation,
                    'Loss': loss,
                    'CRF': crf,
                    'Metric': metric_name,
                    'Value': mean_metric_value
                })
    else:
        print(f"Unknown metrics type for model {model_name}")

# Create a DataFrame
df = pd.DataFrame(all_data)

# Pivot the DataFrame
pivot_table = df.pivot_table(
    index=['Architecture', 'Augmentation', 'Loss', 'CRF'],
    columns='Metric',
    values='Value'
).reset_index()

# Combine factors into a single index
pivot_table['Model Config'] = pivot_table.apply(
    lambda row: f"{row['Architecture']} | {row['Augmentation']} | {row['Loss']} | {row['CRF']}",
    axis=1
)

# Set the index to 'Model Config' and select the metrics columns
heatmap_data = pivot_table.set_index('Model Config')[['Dice', 'Hausdorff']]

# Plot the heatmap
# Determine the figure height
fig_height = max(6, len(heatmap_data) * 0.5)  # Adjust as needed
plt.figure(figsize=(12, fig_height))
sns.heatmap(
    heatmap_data,
    annot=True,
    fmt=".2f",
    cmap='viridis',
    cbar_kws={'label': 'Metric Value'}
)
plt.title('Model Performance Heatmap')
plt.xlabel('Metric')
plt.ylabel('Model Configuration')
plt.tight_layout()
plt.savefig('model_performance_heatmap.png', dpi=300)
plt.show()

# Convert categorical variables
df['Architecture'] = df['Architecture'].astype('category')
df['Augmentation'] = df['Augmentation'].astype('category')
df['Loss'] = df['Loss'].astype('category')
df['CRF'] = df['CRF'].astype('category')

# Create a FacetGrid
g = sns.catplot(
    data=df,
    x='Augmentation',
    y='Value',
    hue='Loss',
    col='Architecture',
    row='Metric',
    kind='bar',
    height=4,
    aspect=1.5,
    palette='Set2'
)

g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('Model Performance by Augmentation and Loss Function')

# Adjust labels
g.set_axis_labels("Augmentation", "Metric Value")
g.add_legend(title='Loss Function')

plt.savefig('model_performance_facetgrid.png', dpi=300)
plt.show()


# Boxplots for losses and augs

In [ ]:

# Load the metrics
metrics_focal = pickle.load(open("3D_focal_loss_10-23(15.08)/metrics3D.pkl", "rb"))
metrics_tversky = pickle.load(open("3D_tversky_loss_10-23(14.45)/metrics3D.pkl", "rb"))
metrics_dice = pickle.load(open("3D_dice_loss_10-23(14.52)/metrics3D.pkl", "rb"))
metrics_combined = pickle.load(open("3D_combined_loss_10-23(15.02)/metrics3D.pkl", "rb"))
metrics_ce = pickle.load(open("3D_ce_loss_10-23(15.14)/metrics3D.pkl", "rb"))



metrics_noise = pickle.load(open("3D_noise_10-23(15.31)/metrics3D.pkl", "rb"))
metrics_affine = pickle.load(open("3D_affine_10-23(15.43)/metrics3D.pkl", "rb"))
metrics_elastic = pickle.load(open("3D_elastic_10-23(15.49)/metrics3D.pkl", "rb"))
metrics_combined = pickle.load(open("3D_combineddata_10-23(15.37)/metrics3D.pkl", "rb"))

In [ ]:



# Combine metrics into a dictionary
# metrics_dict = {
#     'Focal Loss': metrics_focal,
#     'Tversky Loss': metrics_tversky,
#     'Dice Loss': metrics_dice,
#     'Combined Loss': metrics_combined,
#     'Cross Entropy Loss': metrics_ce
# }

metrics_dict = {
    'Noise': metrics_noise,
    'Affine': metrics_affine,
    'Elastic': metrics_elastic,
    'All augmentations': metrics_combined
}

# Define class names
class_names = ['Background', 'Esophagus', 'Heart', 'Trachea', 'Aorta']


# Define a palette for the models
# model_palette = {
#     'Focal Loss': custom_palette['roze'],
#     'Tversky Loss': custom_palette['groen'],
#     'Dice Loss': custom_palette['paars'],
#     'Combined Loss': custom_palette['blue'],
#     'Cross Entropy Loss': custom_palette['orange']
# }
# 
model_palette = {
    'Noise': custom_palette['roze'],
    'Affine': custom_palette['groen'],
    'Elastic': custom_palette['paars'],
    'All augmentations': custom_palette['blue']
}

In [ ]:
# Prepare data for plotting
all_data = []

for model_name, metrics in metrics_dict.items():
    for metric_name in ['3D Dice', '3D Hausdorff distance']:
        metric_values = np.array(metrics[metric_name])  # Shape: (num_samples, num_classes)
        num_samples, num_classes = metric_values.shape
        # Exclude background class (assume class 0 is background)
        values = metric_values[:, 1:]  # Shape: (num_samples, num_classes -1)
        # Compute the average over classes (axis=1)
        avg_values = np.nanmean(values, axis=1)  # Shape: (num_samples,)
        for value in avg_values:
            all_data.append({
                'Model': model_name,
                'Metric': metric_name,
                'Value': value
            })

df = pd.DataFrame(all_data)

# Create violin plots for each metric using custom themes and overlay the mean
for theme in ['light', 'dark']:
    if theme == 'dark':
        set_custom_dark_theme()
    else:
        set_custom_light_theme()
    for metric_name in df['Metric'].unique():
        plt.figure(figsize=(12, 8))
        ax = sns.violinplot(
            data=df[df['Metric'] == metric_name],
            x='Model',
            y='Value',
            palette=model_palette,
            inner=None  # Remove the default inner representation
        )
        
        # Calculate the mean values for each model
        means = df[df['Metric'] == metric_name].groupby('Model')['Value'].mean()
        
        # Get the positions of the x-ticks (models)
        x_positions = []
        labels = []
        for tick, label in zip(ax.get_xticks(), ax.get_xticklabels()):
            x_positions.append(tick)
            labels.append(label.get_text())
        
        # Get the y-axis limits to calculate an appropriate offset
        ymin, ymax = ax.get_ylim()
        y_range = ymax - ymin
        y_offset = y_range * 0.01  # Adjust as needed (e.g., 1% of y-axis range)
        
        # Overlay the mean values on the violin plot
        for x, model_name in zip(x_positions, labels):
            mean_value = means[model_name]
            # Plot the mean point
            ax.scatter(x, mean_value, color='black', marker='o', s=100, zorder=3, label='Mean' if x == x_positions[0] else "")
            # Annotate the mean value, adjust position to avoid overlap
            ax.text(
                x, mean_value + y_offset,  # Move text slightly above the mean point
                f'{mean_value:.2f}',
                color='black',
                ha='center',
                va='bottom',
                fontsize=12
            )
        
        # Add legend only once
        handles, _ = ax.get_legend_handles_labels()
        if handles:
            ax.legend(handles=[handles[0]], labels=['Mean'], loc='upper right')
        
        plt.title(f"{metric_name} across models (average over classes, excl. background)", fontsize=14)
        plt.ylabel(metric_name, fontsize=12)
        plt.xlabel('Model', fontsize=12)
        plt.xticks(rotation=45, fontsize=12)
        plt.tight_layout()
        plt.savefig(f"{metric_name}_average_{theme}.png", dpi=300)
        plt.show()

In [ ]:
# Prepare data for plotting
all_data = []

for model_name, metrics in metrics_dict.items():
    for metric_name in ['3D Dice', '3D Hausdorff distance']:
        metric_values = np.array(metrics[metric_name])  # Shape: (num_samples, num_classes)
        num_samples, num_classes = metric_values.shape
        # Exclude background class (assume class 0 is background)
        for class_idx in range(1, num_classes):
            class_name = class_names[class_idx]
            values = metric_values[:, class_idx]  # Shape: (num_samples,)
            # Exclude Dice scores that are exactly 1.0
            # if metric_name == 'Dice':
            #     values = values[values != 1.0]
            #     values = values[values != 0.0]
            #     
            # if metric_name == 'Hausdorff distance':
            #     values = values[values != 1.0]
            #     values = values[values != 0.0]
                
            for value in values:
                all_data.append({
                    'Model': model_name,
                    'Class': class_name,
                    'Metric': metric_name,
                    'Value': value
                })

df = pd.DataFrame(all_data)


In [ ]:
# Create boxplots for each metric using custom themes
for theme in ['light', 'dark']:
    if theme == 'dark':
        set_custom_dark_theme()
    else:
        set_custom_light_theme()
    for metric_name in df['Metric'].unique():
        plt.figure(figsize=(12, 8))
        sns.boxplot(
            data=df[df['Metric'] == metric_name],
            x='Class',
            y='Value',
            hue='Model',
            palette=model_palette,
            showfliers=False  # Hide outliers for clarity
        )
        plt.title(f"{metric_name} per class for different models", fontsize=14)
        plt.ylabel(metric_name, fontsize=12)
        plt.xlabel('Class', fontsize=12)
        plt.xticks(rotation=45, fontsize=12)
        plt.legend(title='Model', fontsize=12, title_fontsize=12)
        plt.tight_layout()
        plt.savefig(f"{metric_name}_per_class_{theme}.png", dpi=300)
        plt.show()
